In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score
import time
import pywt

In [2]:
rootPath = "./"
# rootPath = "/gdrive/MyDrive/ICALML/"

In [3]:
### Define your algorithm here

def laplacian(frame):
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    val = cv2.Laplacian(image, cv2.CV_64F).var()
    return val


In [4]:
def HWT(img, threshold = 35):
    
    # Convert image to grayscale
    Y = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    
    M, N = Y.shape
    
    # Crop input image to be 3 divisible by 2
    Y = Y[0:int(M/16)*16, 0:int(N/16)*16]
    
    # Step 1, compute Haar wavelet of input image
    LL1,(LH1,HL1,HH1)= pywt.dwt2(Y, 'haar')
    # Another application of 2D haar to LL1
    LL2,(LH2,HL2,HH2)= pywt.dwt2(LL1, 'haar') 
    # Another application of 2D haar to LL2
    LL3,(LH3,HL3,HH3)= pywt.dwt2(LL2, 'haar')
    
    # Construct the edge map in each scale Step 2
    E1 = np.sqrt(np.power(LH1, 2)+np.power(HL1, 2)+np.power(HH1, 2))
    E2 = np.sqrt(np.power(LH2, 2)+np.power(HL2, 2)+np.power(HH2, 2))
    E3 = np.sqrt(np.power(LH3, 2)+np.power(HL3, 2)+np.power(HH3, 2))
    
    M1, N1 = E1.shape

    # Sliding window size level 1
    sizeM1 = 8
    sizeN1 = 8
    
    # Sliding windows size level 2
    sizeM2 = int(sizeM1/2)
    sizeN2 = int(sizeN1/2)
    
    # Sliding windows size level 3
    sizeM3 = int(sizeM2/2)
    sizeN3 = int(sizeN2/2)
    
    # Number of edge maps, related to sliding windows size
    N_iter = int((M1/sizeM1)*(N1/sizeN1))
    
    Emax1 = np.zeros((N_iter))
    Emax2 = np.zeros((N_iter))
    Emax3 = np.zeros((N_iter))
    
    
    count = 0
    
    # Sliding windows index of level 1
    x1 = 0
    y1 = 0
    # Sliding windows index of level 2
    x2 = 0
    y2 = 0
    # Sliding windows index of level 3
    x3 = 0
    y3 = 0
    
    # Sliding windows limit on horizontal dimension
    Y_limit = N1-sizeN1
    
    while count < N_iter:
        # Get the maximum value of slicing windows over edge maps 
        # in each level
        Emax1[count] = np.max(E1[x1:x1+sizeM1,y1:y1+sizeN1])
        Emax2[count] = np.max(E2[x2:x2+sizeM2,y2:y2+sizeN2])
        Emax3[count] = np.max(E3[x3:x3+sizeM3,y3:y3+sizeN3])
        
        # if sliding windows ends horizontal direction
        # move along vertical direction and resets horizontal
        # direction
        if y1 == Y_limit:
            x1 = x1 + sizeM1
            y1 = 0
            
            x2 = x2 + sizeM2
            y2 = 0
            
            x3 = x3 + sizeM3
            y3 = 0
            
            count += 1
        
        # windows moves along horizontal dimension
        else:
                
            y1 = y1 + sizeN1
            y2 = y2 + sizeN2
            y3 = y3 + sizeN3
            count += 1
    
    # Step 3
    EdgePoint1 = Emax1 > threshold;
    EdgePoint2 = Emax2 > threshold;
    EdgePoint3 = Emax3 > threshold;
    
    # Rule 1 Edge Pojnts
    EdgePoint = EdgePoint1 + EdgePoint2 + EdgePoint3
    
    n_edges = EdgePoint.shape[0]
    
    # Rule 2 Dirak-Structure or Astep-Structure
    DAstructure = (Emax1[EdgePoint] > Emax2[EdgePoint]) * (Emax2[EdgePoint] > Emax3[EdgePoint]);
    
    # Rule 3 Roof-Structure or Gstep-Structure
    
    RGstructure = np.zeros((n_edges))

    for i in range(n_edges):
    
        if EdgePoint[i] == 1:
        
            if Emax1[i] < Emax2[i] and Emax2[i] < Emax3[i]:
            
                RGstructure[i] = 1
                
    # Rule 4 Roof-Structure
    
    RSstructure = np.zeros((n_edges))

    for i in range(n_edges):
    
        if EdgePoint[i] == 1:
        
            if Emax2[i] > Emax1[i] and Emax2[i] > Emax3[i]:
            
                RSstructure[i] = 1

    # Rule 5 Edge more likely to be in a blurred image 

    BlurC = np.zeros((n_edges));

    for i in range(n_edges):
    
        if RGstructure[i] == 1 or RSstructure[i] == 1:
        
            if Emax1[i] < threshold:
            
                BlurC[i] = 1                        
        
    # Step 6
    Per = np.sum(DAstructure)/np.sum(EdgePoint)
    
    # Step 7
    if (np.sum(RGstructure) + np.sum(RSstructure)) == 0:
        
        BlurExtent = 100
    else:
        BlurExtent = np.sum(BlurC) / (np.sum(RGstructure) + np.sum(RSstructure))
    
    return BlurExtent

In [5]:
## Define dictionary of algo with name
algorithms = {'laplacian':laplacian,
              'HWT':HWT}

In [6]:
def processVideo(file,algo):
    value = []
    cap = cv2.VideoCapture(file)
    while(1):
        ret,frame = cap.read()
        if ret:
            val = algorithms[algo](frame)     
            value.append(val)
        else:
            break
    return np.min(value),np.mean(value),np.max(value),np.std(value)

In [7]:
## Get features
def getFeatures(data,types = 'laplacian'):
    X_data = []
    y_data = []
    time_logs = []
    for items in zip(data["Filename"],data["Label"]):
        print("Processing video:",items[0])
        start = time.time()
        res = processVideo(items[0],types)
        end = time.time()
        X_data.append([res[0],res[1],res[2],res[3]])
        y_data.append(items[1])
        time_logs.append([end-start])
    return X_data,y_data,time_logs

In [8]:
def trainModel(data,label):
    clf = make_pipeline(SVC(kernel = 'linear', gamma='auto'))
    clf.fit(data, label)
    return clf

def trainModelMulti(data,label):
    clf = make_pipeline(OneVsRestClassifier(SVC(kernel = 'linear', gamma='auto')))
    clf.fit(data, label)
    return clf

def testModel(model,data,label):
    y_pred = model.predict(data)
    pr = precision_score(label,y_pred)    
    re = recall_score(label,y_pred)
    f1 = f1_score(label,y_pred) 
    acc = balanced_accuracy_score(label,y_pred)
    return {'precision':pr,'recall':re,"F1":f1,"acc":acc}

def testModelMulti(model,data,label):
    y_pred = model.predict(data)
    pr = precision_score(label,y_pred,average=None)    
    re = recall_score(label,y_pred,average=None)
    f1 = f1_score(label,y_pred,average=None)  
    return {'precision':pr,'recall':re,"F1":f1}

In [9]:
train = pd.read_excel(rootPath + "train.xlsx")
test = pd.read_excel(rootPath + "test.xlsx")

In [ ]:
## Train Data
X_train,y_train,train_logs = getFeatures(train,'laplacian')
pd.DataFrame(X_train,columns=["min","mean","max","std"]).to_csv(rootPath + "X_train_laplacian.csv",index=None)
pd.DataFrame(y_train,columns=["label"]).to_csv(rootPath + "y_train_laplacian.csv",index=None)

In [ ]:
## Test Data
X_test,y_test,test_logs = getFeatures(test,'laplacian')
pd.DataFrame(X_test,columns=["min","mean","max","std"]).to_csv(rootPath + "X_test_laplacian.csv",index=None)
pd.DataFrame(y_test,columns=["label"]).to_csv(rootPath + "y_test_laplacian.csv",index=None)
pd.DataFrame([train_logs,test_logs]).to_csv(rootPath + "time_laplacian.csv",index=None)

In [27]:
X_train = pd.read_csv(rootPath + "X_train_laplacian.csv")
X_test = pd.read_csv(rootPath + "X_test_laplacian.csv")
y_train = pd.read_csv(rootPath + "y_train_laplacian.csv").values.reshape(-1)
y_test = pd.read_csv(rootPath + "y_test_laplacian.csv").values.reshape(-1)

In [28]:
## Make binary labels
y_train1 = np.array(y_train)
y_train1[y_train1 > 0] = 1

y_test1 = np.array(y_test)
y_test1[y_test1 > 0] = 1

In [29]:
model = trainModel(X_train,y_train1)
testModel(model,X_test,y_test1)

{'precision': 0.8709677419354839,
 'recall': 0.9642857142857143,
 'F1': 0.9152542372881356,
 'acc': 0.6964285714285714}

In [32]:
model.predict(X_test)

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
       1.])

In [31]:
### HWT
## Train Data
# X_train,y_train,train_logs = getFeatures(train,'HWT')
# pd.DataFrame(X_train,columns=["min","mean","max","std"]).to_csv(rootPath + "X_train_hwt.csv",index=None)
# pd.DataFrame(y_train,columns=["label"]).to_csv(rootPath + "y_train_hwt.csv",index=None)
# # ## Test Data
# X_test,y_test,test_logs = getFeatures(test,'HWT')
# pd.DataFrame(X_test,columns=["min","mean","max","std"]).to_csv(rootPath + "X_test_hwt.csv",index=None)
# pd.DataFrame(y_test,columns=["label"]).to_csv(rootPath + "y_test_hwt.csv",index=None)
# pd.DataFrame([train_logs,test_logs]).to_csv(rootPath + "time_hwt.csv",index=None)
## Save features
X_train = pd.read_csv(rootPath + "X_train_hwt.csv")
X_test = pd.read_csv(rootPath + "X_test_hwt.csv")
y_train = pd.read_csv(rootPath + "y_train_hwt.csv").values.reshape(-1)
y_test = pd.read_csv(rootPath + "y_test_hwt.csv").values.reshape(-1)
## Make binary labels
y_train1 = np.array(y_train)
y_train1[y_train1 > 0] = 1
y_test1 = np.array(y_test)
y_test1[y_test1 > 0] = 1
## Train test SVM model
model = trainModel(X_train,y_train1)
testModel(model,X_test,y_test1)

{'precision': 0.8709677419354839,
 'recall': 0.9642857142857143,
 'F1': 0.9152542372881356,
 'acc': 0.6964285714285714}

In [10]:
## Check all methods:
output = []
methods = ["laplacian","fft","hwt","modified_laplacian"]
for ext in methods:
    X_train = pd.read_csv(rootPath + "X_train_" + ext+ ".csv")
    X_test = pd.read_csv(rootPath + "X_test_" + ext + ".csv")
    y_train = pd.read_csv(rootPath + "y_train_" + ext + ".csv").values.reshape(-1)
    y_test = pd.read_csv(rootPath + "y_test_" + ext + ".csv").values.reshape(-1)
    ## Make binary labels
    y_train1 = np.array(y_train)
    y_train1[y_train1 > 0] = 1
    y_test1 = np.array(y_test)
    y_test1[y_test1 > 0] = 1
    
    data = X_train.iloc[:,:]
    label = y_train1
    ## Train test SVM model
    model = make_pipeline(SVC(kernel = 'linear', gamma='auto'))
    model.fit(data, label)
    
    data = X_test.iloc[:,:]
    label = y_test1
    val = testModel(model,data,label)
    output.append(val)
    print(ext,val)

laplacian {'precision': 0.8709677419354839, 'recall': 0.9642857142857143, 'F1': 0.9152542372881356, 'acc': 0.6964285714285714}
fft {'precision': 0.8, 'recall': 1.0, 'F1': 0.888888888888889, 'acc': 0.5}
hwt {'precision': 0.8709677419354839, 'recall': 0.9642857142857143, 'F1': 0.9152542372881356, 'acc': 0.6964285714285714}
modified_laplacian {'precision': 0.8, 'recall': 1.0, 'F1': 0.888888888888889, 'acc': 0.5}


In [40]:
pd.DataFrame(output,index=methods).to_csv("result/svm.csv")

In [42]:
X_train.iloc[:,0]

0     124.596774
1     122.967120
2     120.067307
3     118.074145
4     117.785813
         ...    
85    124.917373
86    124.593396
87    123.607062
88    122.272332
89    121.901565
Name: min, Length: 90, dtype: float64

In [ ]:
## Multiclass 
from sklearn.preprocessing import OneHotEncoder
from sklearn.multiclass import OneVsRestClassifier
encoder = OneHotEncoder()
model = trainModelMulti(X_train,encoder.fit_transform(y_train).toarray())
testModelMulti(model,X_test,encoder.fit_transform(y_test).toarray())